In [3]:
from sklearn.model_selection import train_test_split
import json
import pandas as pd
from sklearn import preprocessing, model_selection, feature_extraction, naive_bayes, metrics

## Naive Bayes

- use the tfidf and naive bayes to split the dataset

In [4]:
# Json to csv
def json2df(json_obj):
    
    ls = []
    for row in json_obj:
        ls.append([
            row['title'],
            row['content'],
            row['author'],
            row['brand_id'],
            row['date'],
            row['url']
        ])
    
    col = [
            'title',
            'content',
            'author',
            'date',
            'url',
            'brand_id'
        ]
    
    return pd.DataFrame(ls, columns=col)

In [5]:
# Import dataset

## Read news
with open('./data/real_news_dump.json') as file:
    real_news = json.load(file)

real_news_df = json2df(real_news)

## Fake news
with open('./data/fake_news_dump.json') as file:
    fake_news = json.load(file)
    
fake_news_df = json2df(fake_news)

In [6]:
# Read news test
real_news_df.head()

,title,content,author,date,url,brand_id
0,謠傳碼頭公園有確診案例 調查局約談網友到案,（中央社記者蕭博文台北4日電）調查局今天表示，網路近期謠傳高雄港63號碼頭、台北青年公園出現...,中央社,9,2020-03-04,https://www.nownews.com/news/20200304/3968815/
1,好暖！高醫返國關懷門診 醫護人員進駐機場檢疫站守國門,武漢肺炎國際間疫情不斷擴大，包含韓國、義大利等多個國家被列入我國的旅遊疫情警告地區。高雄醫學...,記者陳美嘉／高雄報導,9,2020-03-04,https://www.nownews.com/news/20200304/3968420/
2,湯景華縱火奪6命遭判死 最高法院發回更審,（中央社記者蕭博文台北4日電）男子湯景華因細故縱火，導致翁家6人身亡，一審、二審、更一審皆被...,中央社,9,2020-03-04,https://www.nownews.com/news/20200304/3968749/
3,更生人車庫包水餃發跡 實現店面網路販售,（中央社記者劉世怡台北4日電）更生人郭珈瑋出獄後，在車庫包手工水餃起家，再透過圓夢創業小額貸...,中央社,9,2020-03-04,https://www.nownews.com/news/20200304/3968725/
4,黑心藥局囤貨 桃園市調處查獲97萬片醫療口罩,武漢肺炎疫情造成全台口罩慌，居然有不肖業者大量囤貨伺機牟利﹔調查局桃園巿調查處日前接獲情資，...,記者李春台 / 桃園報導,9,2020-02-06,https://www.nownews.com/news/20200206/3920699/


In [7]:
# Fake news test
fake_news_df.head()

,title,content,author,date,url,brand_id
0,今年下半年，因病去世的川普弟弟Robert Trump，今天被發現他投票給了拜登,,事實查核中心,19,2020-11-16,
1,最新超級爆料 ! 美國各州的選票點都在選前安設了秘密監控錄像 所有破壞選舉的罪行 都逃不了朱...,,事實查核中心,19,2020-11-13,
2,每天吃一顆血管清道夫-洋蔥，防治心臟病，血管清得超乾淨...吃洋蔥能增加骨質密度、防癌症、防...,,事實查核中心,19,2020-11-13,
3,公共汽車司機拒絕打開車門，直到他戴上口罩。 最終他戴上口罩，司機打開車門，他則向司機開槍.....,,事實查核中心,19,2020-11-13,
4,美軍已經接管了台灣的空戰指揮系統,,事實查核中心,19,2020-11-12,


## Split dataset

In [8]:
# label real dataset
real_label = [True]*len(real_news_df)
real_news_df['label'] = real_label

In [9]:
real_news_df.head()

,title,content,author,date,url,brand_id,label
0,謠傳碼頭公園有確診案例 調查局約談網友到案,（中央社記者蕭博文台北4日電）調查局今天表示，網路近期謠傳高雄港63號碼頭、台北青年公園出現...,中央社,9,2020-03-04,https://www.nownews.com/news/20200304/3968815/,True
1,好暖！高醫返國關懷門診 醫護人員進駐機場檢疫站守國門,武漢肺炎國際間疫情不斷擴大，包含韓國、義大利等多個國家被列入我國的旅遊疫情警告地區。高雄醫學...,記者陳美嘉／高雄報導,9,2020-03-04,https://www.nownews.com/news/20200304/3968420/,True
2,湯景華縱火奪6命遭判死 最高法院發回更審,（中央社記者蕭博文台北4日電）男子湯景華因細故縱火，導致翁家6人身亡，一審、二審、更一審皆被...,中央社,9,2020-03-04,https://www.nownews.com/news/20200304/3968749/,True
3,更生人車庫包水餃發跡 實現店面網路販售,（中央社記者劉世怡台北4日電）更生人郭珈瑋出獄後，在車庫包手工水餃起家，再透過圓夢創業小額貸...,中央社,9,2020-03-04,https://www.nownews.com/news/20200304/3968725/,True
4,黑心藥局囤貨 桃園市調處查獲97萬片醫療口罩,武漢肺炎疫情造成全台口罩慌，居然有不肖業者大量囤貨伺機牟利﹔調查局桃園巿調查處日前接獲情資，...,記者李春台 / 桃園報導,9,2020-02-06,https://www.nownews.com/news/20200206/3920699/,True


In [10]:
# label fake dataset
fake_label = [False]*len(fake_news_df)
fake_news_df['label'] = fake_label

In [11]:
fake_news_df.head()

,title,content,author,date,url,brand_id,label
0,今年下半年，因病去世的川普弟弟Robert Trump，今天被發現他投票給了拜登,,事實查核中心,19,2020-11-16,,False
1,最新超級爆料 ! 美國各州的選票點都在選前安設了秘密監控錄像 所有破壞選舉的罪行 都逃不了朱...,,事實查核中心,19,2020-11-13,,False
2,每天吃一顆血管清道夫-洋蔥，防治心臟病，血管清得超乾淨...吃洋蔥能增加骨質密度、防癌症、防...,,事實查核中心,19,2020-11-13,,False
3,公共汽車司機拒絕打開車門，直到他戴上口罩。 最終他戴上口罩，司機打開車門，他則向司機開槍.....,,事實查核中心,19,2020-11-13,,False
4,美軍已經接管了台灣的空戰指揮系統,,事實查核中心,19,2020-11-12,,False


In [12]:
compact_df = pd.concat([real_news_df, fake_news_df], ignore_index=True)

In [13]:
# X, Y
X = compact_df['title']
y = compact_df['label']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2)

## Tfidf vector

In [15]:
tfidf_vec = feature_extraction.text.TfidfVectorizer()
tfidf_vec.fit(X_train)

x_train_tfidf = tfidf_vec.transform(X_train)
x_test_tfidf = tfidf_vec.transform(X_test)

In [16]:
from sklearn import naive_bayes

In [17]:
bnb = naive_bayes.BernoulliNB()
bnb.fit(x_train_tfidf, y_train)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [18]:
# Predict
y_pred = bnb.predict(x_test_tfidf)
print('accuracy: {}'.format(metrics.accuracy_score(y_test, y_pred)))

accuracy: 0.6157517899761337


# Analysis

In [20]:
print(metrics.classification_report(y_test, y_pred, target_names=['False', 'True']))

              precision    recall  f1-score   support

       False       0.00      0.00      0.00       161
        True       0.62      1.00      0.76       258

    accuracy                           0.62       419
   macro avg       0.31      0.50      0.38       419
weighted avg       0.38      0.62      0.47       419



/home/nober/Py3.6m/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [23]:
TP, TN, FP, FN = ([], [], [], [])
for i in range(len(y_pred)):
    # TP
    if y_test.iloc[i] == True and y_pred[i] == True:
        TP.append(i)
    # TN
    if y_test.iloc[i] == False and y_pred[i] == False:
        TN.append(i)
    # FP
    if y_test.iloc[i] == True and y_pred[i] == False:
        FP.append(i)
    # FN
    if y_test.iloc[i] == False and y_pred[i] == True:
        FN.append(i)
print('TP: {}, TN: {}, FP: {}, FN:{}'.format(len(TP)/len(y_pred), len(TN)/len(y_pred), len(FP)/len(y_pred), len(FN)/len(y_pred)))

TP: 0.6157517899761337, TN: 0.0, FP: 0.0, FN:0.38424821002386633
